<a href="https://colab.research.google.com/github/yankchina/ColabNotebook/blob/main/LearningPythonAsyncio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning Python Asyncio

Python 异步处理是我目前许多项目中最常见的问题，我还是希望能够用 asyncio 而不是 Multithreading。这样我可以用 `async` + `await` 来写更优雅的代码，并且能够带来更灵活的架构。同时我也想通过这样的方式来做具体事情。


## 学习 Lynn Root 的讲座

我还是希望能从一些实际代码入手来进行学习。

于是我找到 Lynn Root 在 2019 Euro Python Conference 上做的 Lecture

- 视频：[Lynn Root - Advanced asyncio: Solving Real-world Production Problems](https://www.youtube.com/watch?v=sW76-pRkZk8&list=LL&index=1&t=1545s&ab_channel=EuroPythonConference)
- 代码：[https://github.com/econchick/mayhem](https://github.com/econchick/mayhem)

我按照她的讲座内容逐个部分做解读。

这个项目的名称叫做 `mayhem` ，其意思是 混乱。这个与异步处理的内涵是相通的。（2021-12-29 理解）



### Part01

我还是用手动的方式，逐行将 代码写入到 Colab Notebook 中

In [1]:
!pip install attrs